In [74]:
import polars as pl

In [75]:
dfOutputs = pl.read_csv("../data/outputs.csv")

In [76]:
import re

def matchDatetime(text: str):
    pattern = (
        r'(\d{1,2}(?:\s*\.\s*00)?)'     # Day (e.g., 15 or 11.00)
        r'(?:\s?\.*?\,?\s?)'                # Separator (dot, comma or space)
        r'(\d{1,2})'                    # Month
        r'(?:\s?\.*?\,?\s?)'                # Separator (dot, comma or space)
        r'(\d{2,4})'                    # Year
        r'\s*(?:-\s*)?'                 # spaces and Optional dash separator
        r'(\d{1,2}\s?[.:,]?\s?\d{2})'    # Time (with optional separator)
    )

    matches = re.search(pattern, text)

    dateString = None
    timeString = None

    try:
        if matches:
            day = matches.group(1).strip().replace(" ", "")
            month = matches.group(2).strip()
            year = matches.group(3).strip()

            # fix some dates that are like 13.00
            if day.find(".00") != -1:
                day = day.replace(".00", "")

            # fix two digit years
            if len(year) == 2:
                year = "20" + year

            # add leading zeroes where needed
            if len(day) == 1:
                day = "0" + day
            if len(month) == 1:
                month = "0" + month

            dateString = day + '.' + month + '.' + year
            
            timeString = matches.group(4).strip()

        
            timeString = timeString.replace(".", ":").replace("..", ":").replace(",", ":").replace(" ", "")
        
        # check if separator exists, if not add, to change time like 1300 to 13:00
        if timeString.find(":") == -1:
            timeString = timeString[:len(timeString)-2] + ":" + timeString[len(timeString)-2:]
    except:
        pass

    return dateString, timeString

In [77]:
from datetime import datetime

def parseDiffFormats(dateString: str, timeString: str):
    # https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes
    possible_formats = [
        '%d.%m.%Y - %H:%M',
    ]

    date = "0000-00-00"
    time = "00:00"

    try:
        dt_string = dateString.replace(" ", "") + " - " + timeString
    except AttributeError:
        return date, time

    for fmt in possible_formats:
        try:
            dt = datetime.strptime(dt_string, fmt)
            date = dt.strftime('%Y-%m-%d')
            time = dt.strftime('%H:%M')
            break
        except ValueError:
            continue

    return date, time

In [78]:
from datetime import datetime

def extractDatetime(text):
    rawDate, rawTime = matchDatetime(text)

    date = "0000-00-00"
    time = "00:00"

    date, time = parseDiffFormats(rawDate, rawTime)
    
    return {
        "date": date,
        "time": time
    }

In [79]:
dfOutputs = dfOutputs.with_columns([
    (pl.col("content").map_elements(lambda x: extractDatetime(x)['date'], return_dtype=pl.String).alias('date')),
    (pl.col("content").map_elements(lambda x: extractDatetime(x)['time'], return_dtype=pl.String).alias('time'))
])

In [80]:
dfOutputs.head()

year,month,file_name,content,date,time
i64,str,str,str,str,str
2022,"""April""","""TMP-1""","""Prometne informacije 30.…","""2022-04-30""","""18:30"""
2022,"""April""","""TMP-10""","""Prometne informacije 30.…","""2022-04-30""","""13:00"""
2022,"""April""","""TMP-100""","""Prometne informacije 27.…","""2022-04-27""","""06:30"""
2022,"""April""","""TMP-101""","""Prometne informacije 27.…","""2022-04-27""","""06:00"""
2022,"""April""","""TMP-102""","""Prometne informacije 26.…","""2022-04-26""","""20:00"""


In [81]:
dfOutputs.filter(pl.col("date") != "0000-00-00").write_csv("../data/outputs.csv")

In [82]:
dfBadDates = dfOutputs.filter(pl.col("date") == "0000-00-00")

In [83]:
dfBadDates.write_csv("../data/outputs-bad-dates.csv")

dfBadDates.describe()

statistic,year,month,file_name,content,date,time
str,f64,str,str,str,str,str
"""count""",430.0,"""430""","""430""","""430""","""430""","""430"""
"""null_count""",0.0,"""0""","""0""","""0""","""0""","""0"""
"""mean""",2023.930233,null,null,null,null,null
"""std""",0.327123,null,null,null,null,null
"""min""",2022.0,"""April""","""TMP-135""",""" 8.30 Podatki o prometu…","""0000-00-00""","""00:00"""
"""25%""",2024.0,null,null,null,null,null
"""50%""",2024.0,null,null,null,null,null
"""75%""",2024.0,null,null,null,null,null
"""max""",2024.0,"""September""","""TMP9-2024-721""","""promet 8.15 Na štajerski …","""0000-00-00""","""00:00"""


In [84]:
extractDatetime("15. 07 2022         10.00 ")

{'date': '2022-07-15', 'time': '10:00'}